In [1]:
import sys 
import numpy as np # linear algebra
from scipy.stats import randint
import matplotlib.pyplot as plt # this is used for the plot the graph 
%matplotlib inline
import re
from tqdm import notebook
import seaborn as sns
import tensorflow as tf
from scipy import stats
from scipy import optimize
from functools import partial
import pandas as pd
import json
from pandas.io.json import json_normalize
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA #reduce dimension given limited data

The preprocess step can be found at
https://github.com/vbordalo/Communities-Crime/blob/master/Crime_v1.ipynb

In [2]:
attrib = pd.read_csv('attributes.csv', delim_whitespace = True)

In [3]:
crimedata = pd.read_csv('communities.data', names = attrib['attributes'])
print (crimedata.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1994 entries, 0 to 1993
Columns: 128 entries, state to ViolentCrimesPerPop
dtypes: float64(100), int64(2), object(26)
memory usage: 1.9+ MB
None


In [4]:
crimedata.shape

(1994, 128)

In [5]:
data = crimedata.drop(columns=['state','county',
                          'community','communityname',
                          'fold'], axis=1)
data.head()

,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,...,LandArea,PopDens,PctUsePubTrans,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,LemasPctOfficDrugUn,PolicBudgPerPop,ViolentCrimesPerPop
0,0.19,0.33,0.02,0.90,0.12,0.17,0.34,0.47,0.29,0.32,...,0.12,0.26,0.20,0.06,0.04,0.9,0.5,0.32,0.14,0.20
1,0.00,0.16,0.12,0.74,0.45,0.07,0.26,0.59,0.35,0.27,...,0.02,0.12,0.45,?,?,?,?,0.00,?,0.67
2,0.00,0.42,0.49,0.56,0.17,0.04,0.39,0.47,0.28,0.32,...,0.01,0.21,0.02,?,?,?,?,0.00,?,0.43
3,0.04,0.77,1.00,0.08,0.12,0.10,0.51,0.50,0.34,0.21,...,0.02,0.39,0.28,?,?,?,?,0.00,?,0.12
4,0.01,0.55,0.02,0.95,0.09,0.05,0.38,0.38,0.23,0.36,...,0.04,0.09,0.02,?,?,?,?,0.00,?,0.03


In [8]:
data['ViolentCrimesPerPop'].min()

0.0

In [6]:
## deal with missingness
from pandas import DataFrame

data = data.replace('?', np.nan)
feat_miss = data.columns[data.isnull().any()]


In [7]:
feat_miss

Index(['OtherPerCap', 'LemasSwornFT', 'LemasSwFTPerPop', 'LemasSwFTFieldOps',
       'LemasSwFTFieldPerPop', 'LemasTotalReq', 'LemasTotReqPerPop',
       'PolicReqPerOffic', 'PolicPerPop', 'RacialMatchCommPol',
       'PctPolicWhite', 'PctPolicBlack', 'PctPolicHisp', 'PctPolicAsian',
       'PctPolicMinor', 'OfficAssgnDrugUnits', 'NumKindsDrugsSeiz',
       'PolicAveOTWorked', 'PolicCars', 'PolicOperBudg', 'LemasPctPolicOnPatr',
       'LemasGangUnitDeploy', 'PolicBudgPerPop'],
      dtype='object')

In [8]:
data[feat_miss].describe()


,OtherPerCap,LemasSwornFT,LemasSwFTPerPop,LemasSwFTFieldOps,LemasSwFTFieldPerPop,LemasTotalReq,LemasTotReqPerPop,PolicReqPerOffic,PolicPerPop,RacialMatchCommPol,...,PctPolicAsian,PctPolicMinor,OfficAssgnDrugUnits,NumKindsDrugsSeiz,PolicAveOTWorked,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,PolicBudgPerPop
count,1993,319,319,319,319,319,319,319,319,319,...,319,319,319,319,319,319,319,319,319,319
unique,97,38,52,34,55,44,59,75,52,76,...,50,72,30,15,77,63,38,72,3,51
top,0,0.02,0.2,0.98,0.19,0.02,0.14,0.23,0.2,0.78,...,0,0.07,0.03,0.57,0.19,0.02,0.02,0.84,0,0.12
freq,129,80,19,81,17,55,23,15,19,12,...,189,14,48,54,12,27,69,13,126,22


In [9]:
data1=data.drop(columns=feat_miss[1:])

In [10]:
##impute the missing value
# from sklearn.impute import SimpleImputer as Imputer

# imputer = Imputer(missing_values = np.nan, strategy = 'mean')

# imp = imputer.fit(data1[['OtherPerCap']])
# data1[['OtherPerCap']] = imp.transform(data1[['OtherPerCap']])


# delete the observation with missing OtherPerCap
data1comp=data1[~data1.OtherPerCap.isna()].copy()

In [15]:
data1comp.head()

,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,...,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn,ViolentCrimesPerPop
0,0.19,0.33,0.02,0.90,0.12,0.17,0.34,0.47,0.29,0.32,...,0.12,0.42,0.50,0.51,0.64,0.12,0.26,0.20,0.32,0.20
1,0.00,0.16,0.12,0.74,0.45,0.07,0.26,0.59,0.35,0.27,...,0.21,0.50,0.34,0.60,0.52,0.02,0.12,0.45,0.00,0.67
2,0.00,0.42,0.49,0.56,0.17,0.04,0.39,0.47,0.28,0.32,...,0.14,0.49,0.54,0.67,0.56,0.01,0.21,0.02,0.00,0.43
3,0.04,0.77,1.00,0.08,0.12,0.10,0.51,0.50,0.34,0.21,...,0.19,0.30,0.73,0.64,0.65,0.02,0.39,0.28,0.00,0.12
4,0.01,0.55,0.02,0.95,0.09,0.05,0.38,0.38,0.23,0.36,...,0.11,0.72,0.64,0.61,0.53,0.04,0.09,0.02,0.00,0.03


### shuffle the data

In [16]:
train=data1comp.sample(frac=0.6,random_state=223)
test=data1comp.drop(train.index)

# trainy=train.pop('ViolentCrimesPerPop')
# testy=test.pop('ViolentCrimesPerPop')

# train=np.array(train)
# test=np.array(test)
# trainy=np.array(trainy).reshape(-1,1)
# testy=np.array(testy).reshape(-1,1)


In [17]:
dt=np.r_[train,test]

In [18]:
dt_x=dt[:,:-1]

#### pca could be applied, or we could use original space as well

In [19]:
pca=PCA(n_components=15)
dt_x = pca.fit_transform(dt_x)
sum(pca.explained_variance_ratio_)

0.8575395188981959

In [20]:
dt_new=np.c_[dt_x,dt[:,-1]]

### Normalize the space

In [21]:
##alternative to standardize dt besides using sklearn

# def norm_data(train,norm=True):
#     std_train=np.ones(train.shape[1])
#     mu_train=np.zeros(train.shape[1])
#     if(norm):
#         std_train = np.std(train, 0)
#         mu_train=np.mean(train,0)

#     train=(train - np.full(train.shape, mu_train)) / \
#             np.full(train.shape, std_train)

#     return train,mu_train,std_train


# train_x,mutmp,stdtmp=norm_data(dt,norm=True)

In [22]:
scaler=StandardScaler()
scaler.fit(dt_new)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [23]:
dtstd=scaler.transform(dt_new)

### Output data

In [24]:
np.save('crimedt',dtstd)

In [25]:
dtstd.shape

(1993, 16)